In [3]:
import numpy as np
from sklearn.feature_extraction import FeatureHasher
import pandas as pd
import os

# Prepare Training data

In [4]:
# local_raw_data = '../data/ordered_ex1_train.csv'
local_raw_data = 'ordered_ex1_train.csv'

In [5]:
raw_data_df = pd.read_csv(local_raw_data)

In [6]:
raw_data_df

,nct_id,conditions,interventions,number_of_facilities,has_us_facility,country,number_of_sponsors,number_of_sae_subjects,enrollment
0,NCT00000143,hiv infections,ganciclovir,19,t,United States,1,0,61
1,NCT00000143,retinitis,ganciclovir,19,t,United States,1,0,61
2,NCT00000143,cytomegalovirus retinitis,ganciclovir,19,t,United States,1,0,61
3,NCT00000143,hiv infections,cidofovir,19,t,United States,1,0,61
4,NCT00000143,retinitis,cidofovir,19,t,United States,1,0,61
5,NCT00000143,cytomegalovirus retinitis,cidofovir,19,t,United States,1,0,61
6,NCT00000143,hiv infections,ganciclovir triphosphate,19,t,United States,1,0,61
7,NCT00000143,retinitis,ganciclovir triphosphate,19,t,United States,1,0,61
8,NCT00000143,cytomegalovirus retinitis,ganciclovir triphosphate,19,t,United States,1,0,61
9,NCT00000378,depressive disorder,serotonin uptake inhibitors,1,t,United States,2,0,110


In [7]:
# target (percentage of having adverse events) = number_of_sae_subjects / enrollment
raw_data_df['target'] = raw_data_df['number_of_sae_subjects'] / raw_data_df['enrollment']

In [8]:
# drop number_of_sae_subjects and enrollment
raw_data_df.drop(raw_data_df.columns[[7,8]], axis=1, inplace=True)

In [9]:
# convert dataframe to ndarray
raw_data = raw_data_df.as_matrix()

In [10]:
# map has_us_facilities value ('t','f') -> (1, 0)
raw_data[raw_data[:,4] == 't',4]= 1
raw_data[raw_data[:,4] == 'f',4]= 0

In [11]:
raw_data[raw_data[:,7]>1, 7] = 1.0

## Feature hashing

In [25]:
def merge_category(id_cat):
    output = []
    prev_id = None
    tmp = {}
    for nct_id, cat in id_cat:
        if prev_id is None or prev_id == nct_id:
            if prev_id is None:
                prev_id = nct_id
            tmp[cat] = 1
        else:
            output.append(tmp)
            # reset
            tmp = {}
            tmp[cat] = 1
            prev_id = nct_id
    output.append(tmp)
    print('output number of merged unique ids: {}'.format(len(output)))
    return output

### Convert conditions, interventions and countries to hash feature

In [10]:
unique_ids = np.unique(raw_data[:,0])
number_of_uniqueID = unique_ids.shape[0]
print('number of unique nct_ids: {}'.format(number_of_uniqueID))

number of unique nct_ids: 16677


In [12]:
conditions = raw_data[:,[0,1]]
interventions = raw_data[:,[0,2]]
countries = raw_data[:,[0,5]]

In [23]:
# dump the catagory of conditions, interventions and conditions to disk
unique_conditions = np.unique(conditions[:,1])
np.save('conditions_catagories.npy', unique_conditions)

unique_interventions = np.unique(interventions[:,1])
np.save('interventions_catagories.npy', unique_interventions)

unique_countries = np.unique(countries[:,1])
np.save('countries_catagories.npy', unique_countries)

> TODO: draw frequency distribution of the three features

In [12]:
interventions[:,1]

array(['ganciclovir', 'ganciclovir', 'ganciclovir', ..., 'oxycodone',
       'fentanyl', 'adenosine'], dtype=object)

In [26]:
# preprocess the high dimentional features before feed into feature hasher
merged_conditions = merge_category(conditions)
merged_interventions = merge_category(interventions)
merged_countries = merge_category(countries)

output number of merged unique ids: 16677
output number of merged unique ids: 16677
output number of merged unique ids: 16677


In [30]:
number_of_conditions = np.unique(conditions[:,1]).shape[0]
print('number of unique conditions: {}'.format(number_of_conditions))

number_of_interventions = np.unique(interventions[:,1]).shape[0]
print('number of unique interventions: {}'.format(number_of_interventions))

number_of_countries = np.unique(countries[:,1]).shape[0]
print('number of unique countries: {}'.format(number_of_countries))

number of unique conditions: 1909
number of unique interventions: 1846
number of unique countries: 142


In [31]:
# feature hasher
conditions_hasher = FeatureHasher(n_features=int(number_of_conditions * 0.2),
                                                             non_negative=True,input_type='dict')
interventions_hasher = FeatureHasher(n_features=int(number_of_interventions * 0.2),
                                                             non_negative=True,input_type='dict')
countries_hasher = FeatureHasher(n_features=int(number_of_countries),
                                                             non_negative=True,input_type='dict')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


In [35]:
# apply feature hashing
conditions_feature = conditions_hasher.fit_transform(merged_conditions).toarray()
print('conditions_feature shape: {}'.format(conditions_feature.shape))

interventions_feature = interventions_hasher.fit_transform(merged_interventions).toarray()
print('interventions_feature shape: {}'.format(interventions_feature.shape))

countries_feature = countries_hasher.fit_transform(merged_countries).toarray()
print('countries_feature shape: {}'.format(countries_feature.shape))

conditions_feature shape: (16677, 381)
interventions_feature shape: (16677, 369)
countries_feature shape: (16677, 142)


In [43]:
merged_countries[0]

{'United States': 1}

In [47]:
list(countries_hasher.transform([merged_countries[10]]))

[<1x142 sparse matrix of type '<class 'numpy.float64'>'
 	with 2 stored elements in Compressed Sparse Row format>]

### Appending the hashed feature to training data

In [48]:
raw_data[0]

array(['NCT00000143', 'hiv infections', 'ganciclovir', 19, 1,
       'United States', 1, 0.0], dtype=object)

In [49]:
# drop the old conditions, interventions and countries
prev_id = None
new_ids = []
new_data = []
order_arr = []
tmp = []
idx = 0
for data in raw_data:
    cur_id = data[0]
    if prev_id is None or cur_id != prev_id:
        # append id to another array
        new_ids.append(cur_id)
        order_arr.append(idx)
        tmp.append(data[3])
        tmp.append(data[4])
        tmp.append(data[6])
        
        tmp += conditions_feature[idx].tolist()
        tmp += interventions_feature[idx].tolist()
        tmp += countries_feature[idx].tolist()
        
        tmp.append(data[7])
        
        new_data.append(tmp)
        
        # update
        prev_id = cur_id
        tmp = []
        idx += 1

# new data shape: nct_id, number_of_facilities, has_us_facility, 
# number_of_sponsors, conditions_features, interventions_features, contries_features, percentage_of_adverse_event (target)
new_data = np.array(new_data, dtype=np.float32)
order_arr = np.array(order_arr)
new_ids = np.array(new_ids)
print('reconstructed training data shape: {}'.format(new_data.shape))

reconstructed training data shape: (16677, 896)


> note: remove the first nct_id column before send to model

In [50]:
# randomly shuffle the data before categorization
np.random.shuffle(order_arr)
new_data = new_data[order_arr]
new_ids = new_ids[order_arr]

In [51]:
def squeeze(prob_array):
    s = 0.5
    # sample size
    N = len(prob_array)
    return (prob_array * (N - 1) + s) / N

In [52]:
train_size = int(new_data.shape[0] * 0.7)
train_features  = new_data[:train_size, :-1]
train_prob = squeeze(new_data[:train_size, -1])
# squeeze the probability from [0,1] to (0,1)
# ref: https://stats.stackexchange.com/questions/31300/dealing-with-0-1-values-in-a-beta-regression
# ref: http://psycnet.apa.org/doiLanding?doi=10.1037%2F1082-989X.11.1.54
train_label = np.log(np.divide(train_prob, 1 - train_prob))
print('train_features shape: {}'.format(train_features.shape))

validation_size = int(new_data.shape[0] * 0.2)
validation_features = new_data[train_size:train_size + validation_size, :-1]
validation_prob = squeeze(new_data[train_size:train_size + validation_size, -1])
validation_label = np.log(np.divide(validation_prob, 1 - validation_prob))
print('validation_features shape: {}'.format(validation_features.shape))

test_features = new_data[train_size + validation_size:, :-1]
test_prob = squeeze(new_data[train_size:train_size + validation_size, -1])
test_label = np.log(np.divide(test_prob, 1 - test_prob))
print('test_features shape: {}'.format(test_features.shape))

train_features shape: (11673, 895)
validation_features shape: (3335, 895)
test_features shape: (1669, 895)


In [95]:
feature_dim = train_features.shape[1]
print('feature dimention: {}'.format(feature_dim))

feature dimention: 895


### Data conversion

convert the numpy array into recordIO-protobuf or CSV format that can be used by sagemaker linear_learner
model specs: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html

Since algorithms have particular input and output requirements, converting the dataset is also part of the process that a data scientist goes through prior to initiating training. In this particular case, the Amazon SageMaker implementation of Linear Learner takes recordIO-wrapped protobuf, where the data we have today is a pickle-ized numpy array on disk.

Most of the conversion effort is handled by the Amazon SageMaker Python SDK, imported as `sagemaker` below.

In [87]:
import boto3
import re
import sagemaker.amazon.common as smac
import io
from sagemaker import get_execution_role
import sagemaker

bucket = 'sagemaker-tiber-solution'
prefix = 'logistic-regression/yangz5'
s3_train_key = '{}/train/recordio-pb-data'.format(prefix)
s3_train_path = os.path.join('s3://', bucket, s3_train_key)

# Define IAM role
role = get_execution_role()

In [92]:
# training data should be excluding the first nct_id column 
vectors = np.array([t.tolist() for t in train_features]).astype('float32')
labels = np.array([t.tolist() for t in train_label]).astype('float32')
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, vectors, labels)
buf.seek(0)
boto3.resource('s3').Bucket(bucket).Object(s3_train_key).upload_fileobj(buf)

Wrapping the model training setup in a convenience function that takes in the S3 location of the training data, the model hyperparameters that define our training job, and the S3 output path for model artifacts.  Inside the function, we'll hardcode the algorithm container, the number and type of EC2 instances to train on, and the input and output data formats.

In [96]:
from sagemaker.predictor import csv_serializer, json_deserializer
def predictor_from_hyperparams(s3_train_data, hyperparams, output_path):
    """
    Create an Estimator from the given hyperparams, fit to training data, and return a deployed predictor
    """
    # specify algorithm containers and instantiate an Estimator with given hyperparams
    containers = {
        'us-west-2': '174872318107.dkr.ecr.us-west-2.amazonaws.com/linear-learner:latest',
        'us-east-1': '382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:latest',
        'us-east-2': '404615174143.dkr.ecr.us-east-2.amazonaws.com/linear-learner:latest',
        'eu-west-1': '438346466558.dkr.ecr.eu-west-1.amazonaws.com/linear-learner:latest'}
    linear = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
        role,
        train_instance_count=1,
        train_instance_type='ml.m4.xlarge',
        output_path=output_path,
        sagemaker_session=sagemaker.Session())
    linear.set_hyperparameters(**hyperparams)
    # train model
    linear.fit({'train': s3_train_data})
    # deploy a predictor
    linear_predictor = linear.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')
    linear_predictor.content_type = 'text/csv'
    linear_predictor.serializer = csv_serializer
    linear_predictor.deserializer = json_deserializer
    return linear_predictor

#     # return the trained model, for now
#     return linear

In [97]:
# Training a binary classifier with default settings: logistic regression
defaults_hyperparams = {
    'feature_dim': feature_dim,
    'predictor_type': 'regressor',
    'epochs': 20
}
defaults_output_path = 's3://{}/{}/defaults/logisticRegularizedQueezedOutput'.format(bucket, prefix)
defaults_predictor = predictor_from_hyperparams(s3_train_path, defaults_hyperparams, defaults_output_path)

INFO:sagemaker:Creating training-job with name: linear-learner-2018-04-20-05-01-38-054


.............................................................
Docker entrypoint called with argument(s): train
[04/20/2018 05:06:35 INFO 140499912705856] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'10', u'init_bias': u'0.0', u'lr_scheduler_factor': u'0.99', u'num_calibration_samples': u'10000000', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'10', u'lr_scheduler_step': u'100', u'init_method': u'uniform', u'init_sigma': u'0.01', u'lr_scheduler_minimum_lr': u'0.00001', u'target_recall': u'0.8', u'num_models': u'32', u'early_stopping_patience': u'3', u'momentum': u'0.0', u'unbias_label': u'auto', u'wd': u'0.0', u'optimizer': u'adam', u'_tuning_objective_metric': u'', u'early_stopping_tolerance': u'0.001', u'learning_rate': u'auto', u'_kvstore': u'auto', u'normalize_data': u'true', 

INFO:sagemaker:Creating model with name: linear-learner-2018-04-20-05-07-47-009


===== Job Complete =====
Billable seconds: 162


INFO:sagemaker:Creating endpoint with name linear-learner-2018-04-20-05-01-38-054


-----------------------------------------------------------------------!

# Evaluation Process

In [112]:
import math
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

In [133]:
from sagemaker.predictor import csv_serializer, json_deserializer
defaults_predictor.content_type = 'text/csv'
defaults_predictor.serializer = csv_serializer
defaults_predictor.deserializer = json_deserializer

In [108]:
result = []

for idx, train_line in enumerate(test_features):
    prediction = defaults_predictor.predict(train_line.reshape(1, -1))
    result.append([float(prediction['predictions'][0]['score']), test_label[idx], test_prob[idx]])
#     print('idx: {}, nct_id: {}, prediction: {}, label: {}'.format(idx, train_line[0], prediction['predictions'][0]['score'], train_labels[idx]))

The predictor.predict method call takes one parameter, the input data for which you want the SageMaker Endpoint to provide inference. predict will serialize the input data, and send it in as request to the SageMaker Endpoint by an InvokeEndpoint SageMaker operation. InvokeEndpoint operation requests can be made by predictor.predict, by boto3 SageMaker.runtime client or by AWS CLI.

In [137]:
result_np = np.array(result)

In [110]:
result_np

array([[-3.54129243e+00,  8.80535412e+00,  9.99850094e-01],
       [-4.82159424e+00, -8.80522537e+00,  1.49925036e-04],
       [-5.10873175e+00, -8.80522537e+00,  1.49925036e-04],
       ...,
       [ 3.23691988e+00, -8.80522537e+00,  1.49925036e-04],
       [ 2.62284327e+00,  8.80535412e+00,  9.99850094e-01],
       [ 7.02635765e-01, -1.05961657e+00,  2.57382721e-01]])

In [129]:
# calculate the root mean square error
from sklearn.metrics import mean_squared_error
import math
y_true = result_np[:,2].astype(np.float32)
y_pred = sigmoid(result_np[:,0].astype(np.float32))
math.sqrt(mean_squared_error(y_true, y_pred))

0.5315300539891286

# Invoke Endpoint

In [53]:
def np2csv(arr):
    csv = io.BytesIO()
    np.savetxt(csv, arr, delimiter=',',fmt='%s')
    return csv.getvalue().decode().rstrip()

In [123]:
payload = test_features[1667].reshape(1, -1)
print(payload.shape)

(1, 895)


In [124]:
endpoint_name = 'linear-learner-2018-04-20-05-01-38-054'
import boto3
runtime = boto3.Session().client(service_name='runtime.sagemaker',region_name='us-east-2')
import json
# payload = bytearray(payload)
response = runtime.invoke_endpoint(EndpointName=endpoint_name, \
    ContentType='text/csv', \
    Body=np2csv(payload))
score = json.loads(response['Body'].read())['predictions'][0]['score']
print(score)
sigmoid(score)

2.6228432655334473


0.9323173420423828

In [167]:
import math
# boto3 configuration: http://boto3.readthedocs.io/en/latest/guide/configuration.html
# for local machine, finish 'aws configure' with AWS CLI is sufficient
import boto3
import numpy as np
from sklearn.feature_extraction import FeatureHasher
import os
import io
import json


def np2csv(arr):
    csv = io.BytesIO()
    np.savetxt(csv, arr, delimiter=',',fmt='%s')
    return csv.getvalue().decode().rstrip()

def sigmoid(x):
    
    return 1.0 / (1.0 + math.exp(-x))

def invoke_sagemake_endpoint(input_features, sagemaker_endpoint_name, 
                             number_of_conditions=1909, number_of_interventions=1846, number_of_countries=142):
    """
    send a query to AWS sagemaker endpoint which host the ML model that is already trained.
    
    Args:
        input_features (list): [number_of_facilities(integer), 
                                has_us_facility(integer: 0 or 1), 
                                number_of_sponsors(integer), 
                                [list_of_strings](conditions), 
                                [list_of_strings](interventions), 
                                country(string)]
        sagemaker_endpoint_name (str): The sagemaker endpoint name. (e.g. linear-learner-2018-04-13-19-47-11-302)
        number_of_conditions (int): total number of conditions 
        number_of_conditions (int): total number of interventions
        number_of_conditions (int): total number of countries

    Returns:
        float: The probability that a clinical trial will cause serious adverse reaction, given input features.
    """
    
    
    # read in the conditions, interventions and countries catagories
#     unique_conditions = np.load('conditions_catagories.npy')
#     unique_interventions = np.load('interventions_catagories.npy')
#     unique_countries = np.load('countries_catagories.npy')
    
    # feature hasher
    conditions_hasher = FeatureHasher(n_features=int(number_of_conditions * 0.2),
                                                                 non_negative=True,input_type='dict')
    interventions_hasher = FeatureHasher(n_features=int(number_of_interventions * 0.2),
                                                                 non_negative=True,input_type='dict')
    countries_hasher = FeatureHasher(n_features=int(number_of_countries),
                                                                 non_negative=True,input_type='dict')
    # input feature transform to feature hasher input format
    conditions_dict = {}
    for condition in input_features[3]:
        conditions_dict[condition] = 1
        
    interventions_dict = {}
    for intervention in input_features[4]:
        conditions_dict[condition] = 1
    
    contry_dict = {}
    contry_dict[input_features[5]] = 1
    
    hashed_condition_features = conditions_hasher.transform([conditions_dict]).toarray()
    hashed_intervention_features = conditions_hasher.transform([conditions_dict]).toarray()
    hashed_country_features = conditions_hasher.transform([conditions_dict]).toarray()
    
    
    # feature vector: 
    # number_of_facilities, has_us_facility, number_of_sponsors, 
    # conditions_features, interventions_features, contries_features
    feature_vector = []
    feature_vector.append(input_features[0])
    feature_vector.append(input_features[1])
    feature_vector.append(input_features[2])
    feature_vector += hashed_condition_features[0].tolist()
    feature_vector += hashed_intervention_features[0].tolist()
    feature_vector += hashed_country_features[0].tolist()
    feature_vector = np.array(feature_vector).reshape(1, -1)
    print('feature_vector shape: {}'.format(feature_vector.shape))
    
    # invoke the endpoint
    runtime = boto3.Session().client(service_name='runtime.sagemaker',region_name='us-east-2')
    response = runtime.invoke_endpoint(EndpointName=sagemaker_endpoint_name, \
    ContentType='text/csv', \
    Body=np2csv(payload))
    result = response['Body'].read()
    score = json.loads(result)['predictions'][0]['score']
    # map the score to (0,1) probability using sigmoid functions
    return sigmoid(score)
    

In [153]:
import matplotlib.pyplot as plt
import collections
import operator

In [164]:
conditions_count = collections.Counter(conditions[:,1])
sorted_x = sorted(conditions_count.items(), key=operator.itemgetter(1))
sorted_x.reverse()
sorted_x[:10]

[('diabetes mellitus', 13423),
 ('diabetes mellitus, type 2', 11542),
 ('breast neoplasms', 9079),
 ('lymphoma', 6599),
 ('carcinoma, non-small-cell lung', 6120),
 ('lung neoplasms', 5683),
 ('leukemia', 5564),
 ('arthritis', 5286),
 ('arthritis, rheumatoid', 4627),
 ('hiv infections', 4627)]

In [165]:
interventions_count = collections.Counter(interventions[:,1])
sorted_y = sorted(interventions_count.items(), key=operator.itemgetter(1))
sorted_y.reverse()
sorted_y[:10]

[('antibodies, monoclonal', 6786),
 ('insulin', 3767),
 ('ribavirin', 3759),
 ('metformin', 3698),
 ('insulin, globin zinc', 3483),
 ('rituximab', 3420),
 ('peginterferon alfa-2a', 3381),
 ('interferon-alpha', 3370),
 ('methotrexate', 3368),
 ('cyclophosphamide', 3332)]

In [166]:
countries_count = collections.Counter(countries[:,1])
sorted_z = sorted(countries_count.items(), key=operator.itemgetter(1))
sorted_z.reverse()
sorted_z[:10]

[('United States', 82301),
 ('Canada', 14723),
 ('Germany', 13425),
 ('Italy', 9903),
 ('Spain', 9399),
 ('France', 9299),
 ('United Kingdom', 8861),
 ('Australia', 8579),
 ('Poland', 7635),
 ('Russian Federation', 6784)]

In [168]:
endpoint_name = 'linear-learner-2018-04-20-05-01-38-054'
sample_input_feature = [10, 1, 10, ['diabetes mellitus', 'diabetes mellitus, type 2'], 
                        ['antibodies, monoclonal', 'methotrexate'], 'United States']
invoke_sagemake_endpoint(sample_input_feature, endpoint_name)

feature_vector shape: (1, 1146)


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


0.9323173420423828